## Supermarket Data Thieves


### First we scraped supermarketcheck.de 


In [ ]:
## Import libraries

import pandas as pd
import re
import datetime
import requests
import bs4

In [ ]:
## Create a outer for loop for multiple supermarkets.

supermarket_list = ["aldi-nord", "aldi-sued", "edeka", "kaufland", "rewe", "lidl"]

for i in range(0,len(supermarket_list)):
    
    j = supermarket_list[i]
    
    result_frame = supermarket_scraper(j,True)
    
    if "aldi" not in j:
        
        globals()['frame_%s' % j] = result_frame
        
    else:
        
        globals()['frame_aldi_%s' % i] = result_frame
    

In [ ]:
# As a function


def supermarket_scraper(supermarket_input,print_input):

    ## Scrape supermarket products

    # Initialize empty framelist for dataframe and set supermarket
    
    frame_list = []
    supermarket = supermarket_input
    prints = print_input
    
    if prints:
        print("Current supermarket: ", supermarket)
    # Check how many pages for this supermarket
    
    pageno = 2
    base = "https://www.supermarktcheck.de/"
    url = base + f"{supermarket}/produkte/page:{pageno}"
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    pages = str(soup.find("div", class_="col-sm-6 text-center btn-pagination-title").contents).replace("\\t","").replace("\\nSeite ","".replace("\[",""))
    pattern = r"[2 von ](\d+)"
    pagenumbers = int(re.findall(pattern,pages)[0])
    
    
        # Iterate through all pages for the supermarket #pagenumbers+1
        
    for j in range(1, pagenumbers+1):
        
        # Scrape soup for each page
        
        supermarket = supermarket_input
        pageno = j
        base = "https://www.supermarktcheck.de/"
        url = base + f"{supermarket}/produkte/page:{pageno}"
        r = requests.get(url)
        soup = bs4.BeautifulSoup(r.content, "html.parser")
        
        
        # Get a list of products per page
        
        product_list = soup.find("div", class_="col-sm-8 col-md-9").find_all("div", "supermarketListElement")
        
        
        # List of items per page
        
        item_names = soup.find_all("a", class_="supermarket")
        
        
        # Count how many items per page excluding empty items
        
        counter = 0
        for i in product_list:
            try:
                whatever = i.find("div", class_="col-md-5 col-lg-5").find("strong")
                counter += 1
            except:
                next
                
                
        if prints:
            print("items on page:",counter)
        
        
        # For loop for all products on one page
        
        for i in range(0,counter):
            
            # Initialize a list as empty dataframe row
            
            frame_element = []
            
            
            # Get the item names
            
            supermarket_list = item_names[i].contents[0]
            frame_element.append(supermarket_list)


            # Get the item prices

            price = soup.find_all("div", class_="col-md-5 col-lg-5")
            price = price[i].p.strong.contents[0]
            frame_element.append(price)


            # Get price per unit and unit

            price_per = soup.find_all("div", class_="col-md-5 col-lg-5")
            price_per_item_price = price_per[i].span.contents[0]
            price_per_item_i = price_per[i].small.contents[0]
            frame_element.append(price_per_item_price)
            frame_element.append(price_per_item_i)


            # Get the packet size

            packet_size = soup.find_all("div", class_="col-md-7 col-lg-7")
            packet_size = packet_size[i].find_all("p")[1].contents[0]
            frame_element.append(packet_size)


            # Append the list to the frame list

            frame_list.append(frame_element)

        if prints:
            print("page: ", j)
        
    # Transform the frame list into a dataframe
    
    frame = pd.DataFrame(frame_list,columns=["Name","Price","Price per", "Unit", "Packet Size"])
    frame["Supermarket"] = supermarket
    
    return frame

In [ ]:
## Scrape supermarket products


# Initialize empty framelist for dataframe and set supermarket

frame_list = []
supermarket = "edeka"
prints = False

# Check how many pages for this supermarket

pageno = 2
base = "https://www.supermarktcheck.de/"
url = base + f"{supermarket}/produkte/page:{pageno}"
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, "html.parser")
pages = str(soup.find("div", class_="col-sm-6 text-center btn-pagination-title").contents).replace("\\t","").replace("\\nSeite ","".replace("\[",""))
pattern = r"[2 von ](\d+)"
pagenumbers = int(re.findall(pattern,pages)[0])


# Iterate through all pages for the supermarket

for j in range(1,pagenumbers+1):
    
    # Scrape soup for each page
    
    supermarket = "edeka"
    pageno = j
    base = "https://www.supermarktcheck.de/"
    url = base + f"{supermarket}/produkte/page:{pageno}"
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    
    
    # Get a list of products per page
    
    product_list = soup.find("div", class_="col-sm-8 col-md-9").find_all("div", "supermarketListElement")
    
    
    # List of items per page
    
    item_names = soup.find_all("a", class_="supermarket")
    
    
    # Count how many items per page excluding empty items
    
    counter = 0
    for i in product_list:
        try:
            whatever = i.find("div", class_="col-md-5 col-lg-5").find("strong")
            counter += 1
        except:
            next
            
            
    if prints:
        print("items on page:",counter)
    
    
    # For loop for all products on one page
    
    for i in range(0,counter):
        
        # Initialize a list as empty dataframe row
        
        frame_element = []
        
        
        # Get the item names
        
        supermarket_list = item_names[i].contents[0]
        frame_element.append(supermarket_list)
        
        
        # Get the item prices
        
        price = soup.find_all("div", class_="col-md-5 col-lg-5")
        price = price[i].p.strong.contents[0]
        frame_element.append(price)
        
        
        # Get price per unit and unit
        
        price_per = soup.find_all("div", class_="col-md-5 col-lg-5")
        price_per_item_price = price_per[i].span.contents[0]
        price_per_item_i = price_per[i].small.contents[0]
        frame_element.append(price_per_item_price)
        frame_element.append(price_per_item_i)
        
        
        # Get the packet size
        
        packet_size = soup.find_all("div", class_="col-md-7 col-lg-7")
        packet_size = packet_size[i].find_all("p")[1].contents[0]
        frame_element.append(packet_size)
        
        
        # Append the list to the frame list
        
        frame_list.append(frame_element)
    
    if prints:
        print("page: ", j)
        
# Transform the frame list into a dataframe

frame = pd.DataFrame(frame_list,columns=["Name","Price","Price per", "Unit", "Packet Size"])
frame["Supermarket"] = supermarket

## Scrape the data for Brands

In [ ]:
brandlist = list(soup.find("ul", class_="list-group").find_all("li"))

In [ ]:
pd.DataFrame(full_brand_list).to_csv("brandlist.csv")

In [ ]:
counter = 0

list_init = ["7-days"]

while counter != 52:
    
    # Get last brand on the list and open page
    
    last_brand = list_init[-1]
    print("next page:", last_brand)
    url = f"https://www.supermarktcheck.de/{last_brand}/"
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    
    
    # Get all brands displayed on page and append to list
    
    brandlist = list(soup.find("ul", class_="list-group").find_all("li"))
    
    for i in brandlist:
        
        appender = i.find("a").contents[0]
        
        if "Tchibo" in appender:
            appender = str(appender) + "-1"
            
        list_init.append(appender.rstrip().replace("ö","oe").replace(" ","-").replace("ü","ue").replace("&","-").replace("®",""))
        
    
    # Counter
    
    counter += 1
    print(counter)


list_init

## Category Scraping

In [ ]:
## Import libraries

import pandas as pd
import re
import datetime
import requests
import bs4
import selenium 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options

# 1. Selenium scraping of categories

## 1.1. Get category values and plain text

In [ ]:
supermarket_list = ["aldi-nord", "aldi-sued", "edeka", "kaufland", "rewe", "lidl"]

supermarket = supermarket_list[0]
url = f"https://www.supermarktcheck.de/{supermarket}/sortiment"
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, "html.parser")
d = soup.find_all("div", class_="panel-body")[1].find_all("select", class_="form-control")[1].find_all("option")
d

In [ ]:
category_list = []
value_list = []
for i in d:
    category_list.append(i.contents[0])
    stri = str(i.contents[0])
    value = str(i).replace('<option value="',"").replace('">'," ").replace("</option>","")
    value_list.append(value)

digit_pattern = r"^(\d+\w)"
digit_value_list = []
for i in value_list:
    digiz = re.findall(digit_pattern, i)
    digit_value_list.append(digiz)
    
print(category_list, digit_value_list)

In [ ]:
# Remove the first "all category from both"

category_list = category_list[1:]
digit_value_list = digit_value_list[1:]

In [ ]:
## Selenium detour

In [ ]:
soup = 0
#options = Options()
#options.add_argument("--headless")
#driver = webdriver.Firefox(options=options)
driver = webdriver.PhantomJS()
driver.get("https://www.supermarktcheck.de/aldi-nord/sortiment/")
select = Select(driver.find_element_by_name('cat_id'))

try:
    select.select_by_value(digit_value_list[1][0])
except:
    driver.execute_script("window.scrollBy(0, 150);")
    print("scrolled at 1")



try:
    submit_button = driver.find_elements_by_class_name("submit")
except:
    driver.execute_script("window.scrollBy(0, 150);")
    print("scrolled at 2")
    
try:
    submit_button[0].click()
except:
    driver.execute_script("window.scrollBy(0, 150);")
    print("scrolled at 3")

html = driver.page_source
driver.close()
soup = bs4.BeautifulSoup(html,"html.parser")

# 1. Scraping of categories

In [ ]:
for k in range(0,len(digit_value_list)):
    
    print("product category:", category_list[k])
    
    init_frame = pd.DataFrame(columns=['Name','Price','Price per','Unit','Packet Size','Supermarket','Category'])
    
    for i in range(0,len(supermarket_list)):
    
        j = supermarket_list[i]
        kvalue = digit_value_list[k][0]
        
        result_frame = supermarket_scraper(j,True,kvalue)
        result_frame["Category"] = category_list[k]
        
        frame_list = [init_frame,result_frame]
        init_frame = pd.concat(frame_list)
        
        if "aldi" not in j:
            
            globals()[f'frame_{j}_{kvalue}'] = result_frame
            
        else:
            
            globals()[f'frame_aldi_{i}_{kvalue}'] = result_frame
    
    init_frame.to_csv(f"category_{kvalue}_frame.csv")
    globals()[f"../supermarket_csvs/frame_all_supermarkets_category_{kvalue}"] = init_frame

In [ ]:
frame_all_supermarkets_category_21.to_csv("csv.csv")

# 2. Old code to get the csvs

In [ ]:
## Create a outer for loop for multiple supermarkets.

supermarket_list = ["aldi-nord", "aldi-sued", "edeka", "kaufland", "rewe", "lidl"]

for i in range(0,len(supermarket_list)):
    
    j = supermarket_list[i]
    
    result_frame = supermarket_scraper(j,True)
    
    if "aldi" not in j:
        
        globals()['frame_%s' % j] = result_frame
        
    else:
        
        globals()['frame_aldi_%s' % i] = result_frame
    

In [ ]:
# As a function


def supermarket_scraper(supermarket_input,print_input,category_input):

    ## Scrape supermarket products

    # Initialize empty framelist for dataframe and set supermarket
    
    frame_list = []
    supermarket = supermarket_input
    prints = print_input
    
    if prints:
        print("Current supermarket: ", supermarket)
    # Check how many pages for this supermarket
    
    pageno = 2
    base = "https://www.supermarktcheck.de/"
    url = base + f"{supermarket}/produkte/page:{pageno}/cat_id:{category_input}"
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    pages = str(soup.find("div", class_="col-sm-6 text-center btn-pagination-title").contents).replace("\\t","").replace("\\nSeite ","".replace("\[",""))
    pattern = r"[2 von ](\d+)"
    pagenumbers = int(re.findall(pattern,pages)[0])
    
    
        # Iterate through all pages for the supermarket #pagenumbers+1
        
    for j in range(1, pagenumbers+1):
        
        # Scrape soup for each page
        
        supermarket = supermarket_input
        pageno = j
        base = "https://www.supermarktcheck.de/"
        url = base + f"{supermarket}/produkte/page:{pageno}/cat_id:{category_input}"
        r = requests.get(url)
        soup = bs4.BeautifulSoup(r.content, "html.parser")
        
        
        # Get a list of products per page
        
        product_list = soup.find("div", class_="col-sm-8 col-md-9").find_all("div", "supermarketListElement")
        
        
        # List of items per page
        
        item_names = soup.find_all("a", class_="supermarket")
        
        
        # Count how many items per page excluding empty items
        
        counter = 0
        for i in product_list:
            try:
                whatever = i.find("div", class_="col-md-5 col-lg-5").find("strong")
                counter += 1
            except:
                next
                
                
        if prints:
            print("items on page:",counter)
        
        
        # For loop for all products on one page
        
        for i in range(0,counter):
            
            # Initialize a list as empty dataframe row
            
            frame_element = []
            
            
            # Get the item names
            
            supermarket_list = item_names[i].contents[0]
            frame_element.append(supermarket_list)


            # Get the item prices

            price = soup.find_all("div", class_="col-md-5 col-lg-5")
            price = price[i].p.strong.contents[0]
            frame_element.append(price)


            # Get price per unit and unit

            price_per = soup.find_all("div", class_="col-md-5 col-lg-5")
            price_per_item_price = price_per[i].span.contents[0]
            price_per_item_i = price_per[i].small.contents[0]
            frame_element.append(price_per_item_price)
            frame_element.append(price_per_item_i)


            # Get the packet size

            packet_size = soup.find_all("div", class_="col-md-7 col-lg-7")
            packet_size = packet_size[i].find_all("p")[1].contents[0]
            frame_element.append(packet_size)


            # Append the list to the frame list

            frame_list.append(frame_element)

        if prints:
            print("page: ", j)
        
    # Transform the frame list into a dataframe
    
    frame = pd.DataFrame(frame_list,columns=["Name","Price","Price per", "Unit", "Packet Size"])
    frame["Supermarket"] = supermarket
    frame["Category"] = category_input
    
    return frame

## Then we cleaned the data

In [ ]:
## Import libraries

import pandas as pd
import re
import datetime
import requests
import bs4
import numpy as np

In [ ]:
supermarket_csv = ["aldi_products.csv", "aldisued_products.csv", "edeka_products.csv", "kaufland_products.csv", "rewe_products.csv", "lidl_products.csv"]

In [ ]:
def data_cleaner(supermarket_input):
    
        
    df = pd.read_csv(supermarket_input) 
    
    aldi_df = df
    
    aldi_df["Price"] = aldi_df["Price"].str.replace(" €","").str.replace(",",".")
    aldi_df["Price per"] = aldi_df["Price per"].str.replace(" €","").str.replace(",",".")
    aldi_df["Price"] = pd.to_numeric(aldi_df['Price'], errors='coerce')
    aldi_df["Price per"] = pd.to_numeric(aldi_df['Price per'], errors='coerce')
    
        
    aldi_df["1kg Div"] = np.where(aldi_df["Unit"] == ("Preis pro 1 KG"),(aldi_df["Price per"])/10,aldi_df["Price per"])
    aldi_df["ML Div "] = np.where(aldi_df["Unit"] == ("Preis pro 1 Liter"),(aldi_df["Price per"])/10,aldi_df["Price per"])
    aldi_df["Div_merged"] = 0
    aldi_df["Div_merged"] = np.where(aldi_df["Unit"].str.contains("G"), aldi_df["1kg Div"],0)
    aldi_df["Div_merged2"] = np.where(aldi_df["Unit"].str.contains("Liter","ml"), aldi_df["ML Div "], aldi_df["Div_merged"])
    aldi_df["Div_merged3"] = np.where(aldi_df["Unit"].str.contains("ml"), aldi_df["ML Div "], aldi_df["Div_merged2"])
    aldi_df["Comparable Price"] = np.where(aldi_df["Div_merged3"] == 0, aldi_df["Price per"], aldi_df["Div_merged3"])
    
    
    dropped = ["Unnamed: 0","1kg Div","ML Div ","Div_merged","Div_merged2","Div_merged3", "Price per"]
    aldi_df = aldi_df.drop(columns=dropped)
    
    aldi_df["Unit"] = aldi_df["Unit"].str.replace("Preis pro 1 KG","100 Gramm").str.replace("Preis pro 100 Gramm", "100 Gramm").str.replace("Preis pro 1 Liter", "100 ml").str.replace("Preis pro 100 ml", "100 ml")
            
    aldi_df["Packet Size"] = aldi_df["Packet Size"].str.replace("(","").str.replace(")","")
        
        
    
        
    
    return aldi_df
        
        
        
        
    
        
        
        

        
    

    
    
    

In [ ]:
for i in supermarket_csv:
    frame = data_cleaner(i)
    csvpath = "cleaned_" + i
    frame.to_csv(csvpath)

In [ ]:
data_cleaner(supermarket_csv)

## Anaylsis 


In [ ]:
clean = ["supermarket_cleaned_csvs/cleaned_aldi_products.csv",
        "supermarket_cleaned_csvs/cleaned_aldisued_products.csv",
        "supermarket_cleaned_csvs/cleaned_edeka_products.csv",
        "supermarket_cleaned_csvs/cleaned_kaufland_products.csv",
        "supermarket_cleaned_csvs/cleaned_lidl_products.csv",
        "supermarket_cleaned_csvs/cleaned_rewe_products.csv"]
         
supermarket_names = ["aldinord","aldisued","edeka","kaufland","lidl","rewe"]

In [ ]:
# Create one frame with all the comparisons

compare_frame = pd.DataFrame(columns=["Market1","Market1Mean","Market2","Market2Mean","RatioM1/M2"])



# Outer loop: Choose "base" file for merge

# Loop for indices of clean
for i in range(0,len(clean)):
    df1 = pd.read_csv(clean[i])
    
    # Now, inner loop for all other files, same approach
    for j in range(0,len(clean)):
        df2 = pd.read_csv(clean[j])
        
        # Do the merge operation for all
        df_merge = pd.merge(df1,df2,how='inner',on="Name")
        
        
        # Save as .csv. Give meaningful names. Since name list has same order as .csv file list, 
        # we can use the same indices
        
        name1 = supermarket_names[i]
        name2 = supermarket_names[j]
        df_merge.to_csv(f"{name1}_{name2}.csv")
        
        # Calculate means and ratio
        
        mean1 = df_merge["Comparable Price_x"].mean()
        mean2 = df_merge["Comparable Price_y"].mean()        
        
        meanratio = mean1/mean2
        
        
        # Add the row to the above frame
        
        comparerow = [name1, mean1, name2, mean2, meanratio]
        
        compare_frame.append(comparerow)
        
# save compare frame individually

compare_frame.to_csv("compareframe.csv")
